In [2]:
# init imports

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (Dense, Dropout, LSTM, Conv2D, Flatten, Bidirectional, MaxPool3D, Reshape)
import matplotlib.pyplot as plt

In [2]:
def view_midi(np_array):
    np_array = np_array.reshape(np_array.shape[0]*np_array.shape[1], np_array.shape[2])
    midi_array = np.flip(np_array.T, axis=0)
    f = plt.figure()
    f.set_figwidth(20)
    f.set_figheight(10)
    plt.imshow(midi_array, cmap='binary', interpolation='None', aspect="auto")
    plt.show()
    return f

In [3]:
from tensorflow.python.client import device_lib
import sys
import GPUtil

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# print out system information
print ('system version: ', sys.version)
print('tensorflow version: ', tf.__version__)
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print('available devices: ', get_available_devices()) 

# set gpu as device
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
tf.config.set_visible_devices(gpus[0], 'GPU')

# allow gpu growth
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# get gpu info
GPUtil.showUtilization()

system version:  3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]
tensorflow version:  2.10.1
available devices:  ['/device:CPU:0', '/device:GPU:0']
Num GPUs Available:  1
| ID | GPU | MEM |
------------------
|  0 |  6% | 23% |


In [11]:
# model

def my_model():
  model = tf.keras.Sequential()
  
  model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1)))
  model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1)))
  model.add(MaxPool3D(pool_size=(1, 2, 2)))
  model.add(Dropout(0.4))
  
  model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1)))
  model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1)))
  model.add(MaxPool3D(pool_size=(1, 2, 2)))
  model.add(Dropout(0.4))
  
  # model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1)))
  # model.add(Conv2D(filters=128, kernel_size=(2,2), strides=(1,1)))
  # model.add(MaxPool3D(pool_size=(1, 2, 2)))
  # model.add(Dropout(0.4))
  
  model.add(Reshape((28, 128)))
  
  model.add(Bidirectional(LSTM(units=50, activation='tanh', return_sequences=True)))
  model.add(Bidirectional(LSTM(units=50, activation='tanh', return_sequences=True)))
  model.add(Bidirectional(LSTM(units=50, activation='tanh')))  
  model.add(Dropout(0.4))

  model.add(Dense(9, activation='sigmoid', name='output'))
  return model


In [5]:
# loss and optimizer

def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1
  
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost.
    Average (1 - soft-F1) across all labels.
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost

In [12]:
# create new model
LR = 0.000001
INPUT_SHAPE = (None, 1, 64, 16, 1)

model = my_model()
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss=tf.losses.binary_crossentropy,
  metrics=[macro_soft_f1],
)

model.build(INPUT_SHAPE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 1, 63, 15, 128)    640       
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 62, 14, 128)    65664     
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 1, 31, 7, 128)    0         
 3D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 1, 31, 7, 128)     0         
                                                                 
 conv2d_14 (Conv2D)          (None, 1, 30, 6, 128)     65664     
                                                                 
 conv2d_15 (Conv2D)          (None, 1, 29, 5, 128)     65664     
                                                      

In [7]:
# mappings for our own training (9)
KICK = 0
SNARE = 1
HH_CLOSED = 2
HH_OPEN = 3
RIDE = 4
TOM_1 = 5
TOM_2 = 6
TOM_3 = 7
CRASH = 8
NUM_FEATS = 9

In [6]:
# load data and reshape for training
DATA_ID = '_128mel_16frames_0.1sec'

train_feats_path = "data/dataset" + DATA_ID + "/" + "train_feats" + DATA_ID + ".npy"
train_labels_path = "data/dataset" + DATA_ID + "/" + "train_labels" + DATA_ID + ".npy"

test_feats_path = "data/dataset" + DATA_ID + "/" + "test_feats" + DATA_ID + ".npy"
test_labels_path = "data/dataset" + DATA_ID + "/" + "test_labels" + DATA_ID + ".npy"

val_feats_path = "data/dataset" + DATA_ID + "/" + "val_feats" + DATA_ID + ".npy"
val_labels_path = "data/dataset" + DATA_ID + "/" + "val_labels" + DATA_ID + ".npy"

with tf.device('/CPU:0'):
    # load in data
    train_feats_npy = np.load(train_feats_path, allow_pickle=True)
    train_labels_npy = np.load(train_labels_path, allow_pickle=True)
    
    test_feats_npy = np.load(test_feats_path, allow_pickle=True)
    test_labels_npy = np.load(test_labels_path, allow_pickle=True)
    
    val_feats_npy = np.load(val_feats_path, allow_pickle=True)
    val_labels_npy = np.load(val_labels_path, allow_pickle=True)
    
    print ('train feats.shape: ', train_feats_npy.shape)
    print ('train labels.shape: ', train_labels_npy.shape)
    
    print ('test feats.shape: ', test_feats_npy.shape)
    print ('test labels.shape: ', test_labels_npy.shape)
    
    print ('val feats.shape: ', val_feats_npy.shape)
    print ('val labels.shape: ', val_labels_npy.shape)

    train_feats_npy = train_feats_npy.reshape(train_feats_npy.shape[0], 1, train_feats_npy.shape[1], train_feats_npy.shape[2], 1)
    test_feats_npy = test_feats_npy.reshape(test_feats_npy.shape[0], 1, test_feats_npy.shape[1], test_feats_npy.shape[2], 1)
    val_feats_npy = val_feats_npy.reshape(val_feats_npy.shape[0], 1, val_feats_npy.shape[1], val_feats_npy.shape[2], 1)

    print ('train feats.shape: ', train_feats_npy.shape)
    print ('train labels.shape: ', train_labels_npy.shape)
    
    print ('test feats.shape: ', test_feats_npy.shape)
    print ('test labels.shape: ', test_labels_npy.shape)
    
    print ('val feats.shape: ', val_feats_npy.shape)
    print ('val labels.shape: ', val_labels_npy.shape)

train feats.shape:  (594835, 64, 32)
train labels.shape:  (594835, 9)
test feats.shape:  (84464, 64, 32)
test labels.shape:  (84464, 9)
val feats.shape:  (86313, 64, 32)
val labels.shape:  (86313, 9)
train feats.shape:  (594835, 1, 64, 32, 1)
train labels.shape:  (594835, 9)
test feats.shape:  (84464, 1, 64, 32, 1)
test labels.shape:  (84464, 9)
val feats.shape:  (86313, 1, 64, 32, 1)
val labels.shape:  (86313, 9)


In [16]:
# train the network!
EPOCHS = 1
BATCH_SIZE = 1

# Define some callbacks to improve training.
#early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

with tf.device('/GPU:0'):
    model.fit(
        x=train_feats_npy, 
        y=train_labels_npy, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        validation_data=(val_feats_npy, val_labels_npy), 
        callbacks=[reduce_lr]
        )

594835/594835 [==============================] - 15587s 26ms/step - loss: 0.1581 - macro_soft_f1: 0.9904 - val_loss: 0.1520 - val_macro_soft_f1: 0.9893 - lr: 1.0000e-06


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(train_feats_npy, train_labels_npy, batch_size=BATCH_SIZE)
print("test loss, test acc:", results)

In [17]:
# save model

MODEL_ID = 'model_64mel_16frames_0.05sec_1batch_1epochs'
model.save('models/' + MODEL_ID)

INFO:tensorflow:Assets written to: models/model_64mel_16frames_0.05sec_1batch_1epochs\assets


INFO:tensorflow:Assets written to: models/model_64mel_16frames_0.05sec_1batch_1epochs\assets
